## Snapshot Proper Orthogonal Decomposition POD

Author: Julian Lißner

For questions and feedback write a mail to: [lissner@mib.uni-stuttgart.de](mailto:lissner@mib.uni-stuttgart.de)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from numpy.fft import ifft2, fft2

sys.path.extend( ['provided_functions', 'incomplete_functions' ])
import snapshot_pod as pod 
import result_check as check
sys.path.append( '../submodules' )
from general_functions import file_size

## Lossy image compression
- the Schmidt-Eckard Young theorem states:
- given the SVD $ \qquad\underline{\underline V}\, \underline{\sigma} \,\underline{\underline W}^T = \text{svd}(\underline{\underline A})$
- the optimal rank-$k$ approximation w.r.t. the Frobenius and dual norm is given by (underlines omitted for better readability)
 $$  A^{(k)} = \sum_{i=1}^k \sigma_k v_k  w^T_k$$
- the introduced error is then $e^2 = \sum\limits_{i=k+1}^n \sigma_i^2  = || A -  A^{(k)} ||_F^2 $
- with the relative error $e_{\rm rel}^2 = \dfrac{\sum_{i=k+1}^n \sigma_i^2 }{ \sum_{j=1}^n \sigma_j^2} = \dfrac{\sum_{i=k+1}^n \sigma_i^2 }{ ||  A||_F^2}$
- we can store arrays as their singular values and singular vectors<br>
$\quad\blacktriangleright$ the SVD can be used for lossy image compression

---------
__Task:__ Compute the SVD and the truncation threshold for the given image.

In [ ]:
image = np.load( 'data/lena_gray.npz')['arr_0'].copy()
V, sigma, WT #TODO #svd is found in the numpy.linalg package

truncation_threshold = 0.1  #TODO #try out different parameters
info = np.sum( sigma**2 )
for N in range( len(sigma) ):
    error_rel = np.sqrt( np.max( (0, #TODO) )) #np.max to circumvent possible numerical issues
    if error_rel <= #TODO: 
        break
print( 'number of singular values/vectors required for specified precision:', N)

- Note that $N$ is relatively high since image are (usually) high rank
- the approximate image is given as $\widetilde{\underline{\underline A}} =  \widetilde{\underline{\underline V}}\, \widetilde{\underline{\sigma}} \,\widetilde{\underline{\underline W}}^T  $
---------
__Task:__ Truncate the singular vectors/values and reassemble the image

In [ ]:
V_tilde = #TODO
sigma_tilde= sigma[:N]
WT_tilde = #TODO

reconstructed_image = #TODO #recompute the image using the truncated SVD
fig, axes = plt.subplots( 1, 3, figsize=(18,6) )
axes[0].imshow( image, cmap='gray')
axes[1].imshow( reconstructed_image, cmap='gray')
axes[2].semilogy( sigma, label='singular values' )
axes[2].axvline( N, color='k', label='truncation threshold')

titles = [ 'original image', 'reconstructed image', 'truncation criteria' ]
for ax in axes:
    ax.set_title( titles.pop(0))
axes[2].legend()
axes[2].set_xticks( list(axes[2].get_xticks()) + [N] )
axes[2].set_xlim( xmin=0, xmax=len(sigma))
check.image_truncation( N, V_tilde, sigma_tilde, WT_tilde, reconstructed_image, image) 

- the actual relative error is defined as $e_{\rm rel}= \sqrt{ \dfrac{ || A- \tilde A||^2_F}{ || A||^2_F}} $ with the image $A$

---------
__Task:__ Compare the found relative error to the truncation threshold you specified

In [ ]:
n_pixels = np.prod( image.shape)
n_svd = np.sum( [ np.prod( V_tilde.shape),  len(sigma_tilde), np.prod( WT_tilde.shape) ] )
print( 'compression ratio:\t {:1.4f}\n% of storage required:\t{:5.2f}%\nnumber of pixels saved up:\t{:8.0f}(out of {})\n'.format( 
n_pixels/n_svd,  1/( n_pixels/n_svd)*100,  n_pixels-n_svd, n_pixels) )


e_rel = #TODO

print( 'defined relative error: {:3.5f}%'.format( 100*truncation_threshold ) )
print( 'actual relative error:  {:3.5f}%'.format( 100*e_rel ) ) 

--------------
## Reduced basis construction

- a reduced basis can be computed constructed using an SVD or an eigenvalue decomposition
- often the metric $\underline{\underline M}=\underline{\underline I}$ can be set to identity matrix
- generally $\underline{\underline M} \neq \underline{\underline I}$
- in order to achieve model order reduction, the problem dimension should be significantly reduced
- the truncation threshold is given as (similar to the Schmidt Eckard Young theorem)
$$ \delta_N \geq \sqrt{\frac{ \sum_{i=N+1}^n \sigma_i^2}{ \sum_{j=1}^n \sigma_j^2}} =\sqrt{1 - \frac{ \sum_{i=1}^N \sigma_i^2}{ \sum_{j=1}^n \sigma_j^2}} $$

---------
__Task:__ Implement the `truncation` function in 'snapshot_pod.py'.

In [ ]:
check.truncation_implementation()

- the reduced basis can be implemented using the SVD and the snapshot correlation matrix
- both yield the exact same result, except for numerical precision
- if the metric is set to the identity matrix, the implementation is significantly easier
- the RB computation via the snapshot correlation matrix is implemented as:<br>
$ \underline{\underline C}_s = \underline{\underline S}^T\, \underline{\underline M}\,\underline{\underline S} $ <br>
$ \underline{\underline Q}\, \underline{\underline \Sigma}^2\,\underline{\underline Q}^T = \text{eig}\big( \underline{\underline C_s} )$ <br>
$\underline{\underline A} = \widetilde{\underline{\underline Q}}\,\widetilde{\underline{\underline \Sigma}}^{-1} =  \big[ \underline v_1\cdot\frac1{\sigma_1},\quad \underline v_2\cdot\frac1{\sigma_2},\quad \cdot\cdot\cdot, \quad \underline v_N\cdot\frac1{\sigma_N} \big]$<br>
 finally $\underline{\underline B}$ is given as<br>
 $ \underline{\underline B} = \underline{\underline S}\, \underline{\underline A} $

with $\tilde \ast$ denoting the truncated arrays
- Note the square on $\Sigma$ in the eigenvalue decomposition
- also note that numpy.linalg.eig returns the eigenvalues in _ascending_ order

-----
__Task:__ Implement the RB computation in 'snapshot_pod.py' via the eigenvalue decomposition considering a metric.

In [ ]:
check.reduced_basis( pod.correlation_matrix)

- the computation of the reduced basis using the SVD is implemented as:<br>
 $ \underline{\underline M} = \underline{\underline L}\, \underline{\underline L}^T \qquad$ cholesky decomposition<br>
 $ \underline{\underline U}_{\ast} = \underline{\underline L}^T\, \underline{\underline S} $<br>
$ \text{svd}\big( \underline{\underline U}_{\ast}\big) =\underline{\underline V}\, \underline{\underline \Sigma}\, \underline{\underline W}^T $<br>
$ \underline{\underline B} = \underline{\underline L}^{-T}\, \widetilde{\underline{\underline V}} $

---------
__Task:__ Implement the RB computation in 'snapshot_pod.py' via the SVD considering a metric.

In [ ]:
check.reduced_basis( pod.svd_rb)

## Application of the RB for storage efficiency
- similar to image compression, we can use the RB to store large arrays more efficiently
- instead of storing the data, we can store the computed RB and the reduced coefficients $\xi$
- approximate reassembly of the data is given by $\underline{\underline S} \approx \underline{\underline B}\,\underline{\underline{\xi}} = \underline{\underline B}\,\underline{\underline B}^T\, \underline{\underline S}$
- the relative projection error is defined as $P_{\delta} = \sqrt{ \dfrac{ || \underline{\underline A} - \underline{\underline B} \, \underline{\underline B}^T\, \underline{\underline {A}}||_F^2 }{ || \underline{\underline A} ||_F^2 } }\,$
- as image data we will use the so called _2 point correlation function_ of microstructure images
-----
__Task:__ Compute the reduced basis of the `pcf`, store the basis and the reduced coefficients as a file. 

In [ ]:
from rve_functions import compute_pcf

images = np.load( 'data/rve_images.npz')['arr_0']
resolution = (400, 400)
n_image = images.shape[1]
pcf = np.zeros( images.shape)
dim = np.prod( resolution) 
for i in range( n_image): 
    pcf[:,i] = compute_pcf( images[:,i] )

truncation_threshold = 0.01
B = #TODO #use your defined function

xi = #TODO
p_delta = #TODO #relative projection error
print( 'number of eigenmodes required:\t\t\t', B.shape[-1])
print( 'mean relative projection error on the data:\t', p_delta)
print( 'vs the truncation threshold:\t\t\t', truncation_threshold )

np.savez_compressed( 'results/snapshots.npz', pcf )
np.savez_compressed( 'results/reduced_representation.npz', #TODO )



print( 'size of compressed snapshots:\t\t\t', file_size( 'results/snapshots.npz' ))
print( 'size of compressed reduced representation:\t', file_size( 'results/reduced_representation.npz' ) )
del pcf, B, xi 

- the projection error can be reformulated into an efficient (implementation) manner:
 \begin{equation*}P_{\delta} = \sqrt{ \dfrac{ || \underline{\underline A} - \underline{\underline B} \, \underline{\underline B}^T\, \underline{\underline A}||_F^2 }{ || \underline{\underline A} ||_F^2 } }\, \overset{\rm some\, algebra}{======} \sqrt{ 1-\Big(\dfrac{ || \underline\xi||_F }{ || \underline{\underline A} ||_F } }\Big)^2 \end{equation*}
- using the given identities:
    - $|| \underline{\underline A}- \underline{\underline B}||_F^2 = ||\underline{\underline A}||_F^2+ ||\underline{\underline B}||_F^2 - 2 \langle \underline{\underline A}, \underline{\underline B} \rangle_F$
    - $\langle \underline{\underline A}, \underline{\underline B} \rangle_F = \text{trace} ( \underline{\underline A}^T \underline{\underline B} )$
    - $||\underline{\underline A} ||_F^2 = \text{trace} ( \underline{\underline A}^T \underline{\underline A} )$
    - $\text{trace} ( \underline{\underline A} ) = \text{trace} ( \underline{\underline A}^T) $
    - $\text{trace} ( \underline{\underline A}^T \underline{\underline B} ) = \text{trace} ( \underline{\underline A} \,\underline{\underline B}^T )= \text{trace} ( \underline{\underline B}^T \,\underline{\underline A} )= \text{trace} ( \underline{\underline B}\, \underline{\underline A}^T )$
    - $\text{trace} ( \underline{\underline A}\, \underline{\underline B}\, \underline{\underline C} \,) = \text{trace} ( \underline{\underline C}\, \underline{\underline A}\, \underline{\underline B}\, ) = \text{trace} ( \underline{\underline B}\, \underline{\underline C} \,\underline{\underline A} ) $

---
__Task:__ Derive the efficient implementation of the projection error by hand (on paper) and implement it. Also state the major computational speedups attained by it.

In [ ]:
B = np.load(  'results/reduced_representation.npz')['arr_0']
xi = np.load( 'results/reduced_representation.npz')['arr_1'] 
pcf = np.load( 'results/snapshots.npz' )['arr_0']

projection_error = #TODO #efficient implementation
print( 'mean relative projection error with efficient computation:', projection_error )

pcf_reconstructed = #TODO 
assert np.allclose( pcf_reconstructed.shape, (B.shape[0], xi.shape[1]) ), 'reconstruction wrongly implemented'

n_plotting = 4 #TODO #can be adjusted
indices = np.random.randint( 0, xi.shape[1], n_plotting ) 
fig, axes = plt.subplots( n_plotting, 4, figsize=(4*n_plotting,16) )
axes[0,0].set_title( 'image data' )
axes[0,1].set_title( 'true snapshot' )
axes[0,2].set_title( 'reconstructed snapshot' )
axes[0,3].set_title( 'deviation' )
for i in range( n_plotting):
    axes[i,0].imshow( images[:,indices[i] ].reshape( resolution) )
    axes[i,1].imshow( pcf[:,indices[i] ].reshape( resolution) )
    axes[i,2].imshow( pcf_reconstructed[:,indices[i] ].reshape( resolution) )
    handle = axes[i,3].imshow( (pcf[:,indices[i] ] - pcf_reconstructed[:,indices[i] ]).reshape( resolution) )
    plt.colorbar( handle, ax=axes[i,3] )
for ax in axes.flatten():
    ax.axis('off')

-----
__Task:__ VOLUNTARY: implement and plot the development of the projection error over the number of eigenmodes.<br>
You could also compute a basis of size $N = n_s$ to see the full development of the projection error.<br>
Hint: $\xi$ can be precomputed and simply indexed in a loop.

In [ ]:
#TODO...
fig, ax = plt.subplots( figsize=(6,6))
#TODO...
ax.set_xlabel( '# of eigenmodes' )
ax.set_ylabel( r'projection error $P_{\delta}$ [%]' )

Clear up unneccesary space on the hard drive

In [ ]:
import os
os.system( 'rm results/*.npz' )